In [1]:
from autogen_agentchat.agents import AssistantAgent
from autogen_agentchat.conditions import TextMessageTermination
from autogen_agentchat.teams import RoundRobinGroupChat
 
from autogen_agentchat.base import TaskResult
from autogen_agentchat.conditions import ExternalTermination, TextMentionTermination
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient
from autogen_ext.agents.web_surfer import MultimodalWebSurfer
import os

model = os.environ["MODEL_NAME"]
api_key = os.environ["API_KEY"]
base_url = os.environ["LLM_BASE_URL"]

model_client = OpenAIChatCompletionClient(
    model=model,
    base_url=base_url,
    api_key=api_key,
    parallel_tool_calls= False,
    model_info={
        "family": "llama",
        "vision": False,
        "json_output": True,
        "function_calling": True,
    },
)

In [ ]:

# Create a tool for incrementing a number.
def increment_number(number: int) -> int:
    """Increment a number by 1."""
    return number + 1



web_surfer_agent = MultimodalWebSurfer(
    name="WebScraper",
    model_client=model_client,
    downloads_folder="./downloads",  # Folder where downloaded files can be saved
    description="Agent who can scrape web pages and search for information on the web.",
    debug_dir="./debug",             # Directory for saving debug output (optional)
    headless=True,                   # Run the browser in headless mode
    start_page="https://www.google.com",  # The page where the browser should start
    animate_actions=False,           # Do not animate actions by default
    to_save_screenshots=True,        # Save screenshots during navigation (if needed)
    use_ocr=False,                   # Do not use OCR by default
    browser_channel=None,            # Use the default browser channel
    browser_data_dir=None,           # Use the default data directory for the browser
    to_resize_viewport=True,         # Enable viewport resizing if required
    playwright=None,                 # Let the agent initialize its own Playwright instance
    context=None                     # Use the default browser context
)

external_termination = ExternalTermination()
text_termination = TextMentionTermination("APPROVE")
 
assistant_agent = AssistantAgent(
    name="assistant",
    model_client=model_client,
    system_message="You are an advanced AI agent who can handel customer requests critically and communicate with other agents. Respond with 'APPROVE' to when your feedbacks are addressed. Please respond in English for all queries.",
    model_client_stream= True
)

primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant. Don't include additional words or phrase. Simply provide the answer that user wants.",
)


critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    system_message="Provide constructive feedback. Only Respond with 'APPROVE' to when your feedbacks are addressed. Otherwise, don't say 'APPROVE' in your responses. ",
)


In [ ]:
import asyncio

agents = [primary_agent, critic_agent]
web_handler_agents = [primary_agent,assistant_agent, web_surfer_agent ]
team = RoundRobinGroupChat(agents, termination_condition=external_termination | text_termination,)
 
await team.reset()  # Reset the team for a new task.
# async for message in team.run_stream(task="Write a short poem about the fall season."):  # type: ignore
#     if isinstance(message, TaskResult):
#         print("Stop Reason:", message.stop_reason)
#     else:
#         print(message)

 
run = asyncio.create_task(Console(team.run_stream(task="Write a short poem about the fall season.")))

await asyncio.sleep(0.1)


---------- user ----------
Write a short poem about the fall season.


---------- primary ----------
Golden leaves fall from the tree,
Crimson and amber, dancing free.
The air is crisp, the wind is cold,
As nature's final dance begins to unfold.

The scent of woodsmoke fills the air,
As earthy smells and decaying fare,
The trees stand tall, but soon will fade,
Their beauty fleeting, like a summer shade.
[Prompt tokens: 47, Completion tokens: 74]
---------- critic ----------
A nice descriptive poem about fall! However, I think you could use a few more vivid details to make it even more engaging. For example, you might consider adding some specific examples of the types of leaves mentioned in the first line - "Golden maples" or "Crimson oaks", perhaps?

Additionally, you mention that the trees stand tall but will soon fade - is this a subtle allusion to the passing nature of life? If so, it's a theme worthy of further exploration. Consider adding more depth and emotion to your poem by expanding on this idea.

Lastly, while your final line about summer shade

TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=47, completion_tokens=74), metadata={}, content="Golden leaves fall from the tree,\nCrimson and amber, dancing free.\nThe air is crisp, the wind is cold,\nAs nature's final dance begins to unfold.\n\nThe scent of woodsmoke fills the air,\nAs earthy smells and decaying fare,\nThe trees stand tall, but soon will fade,\nTheir beauty fleeting, like a summer shade.", type='TextMessage'), TextMessage(source='critic', models_usage=RequestUsage(prompt_tokens=132, completion_tokens=156), metadata={}, content='A nice descriptive poem about fall! However, I think you could use a few more vivid details to make it even more engaging. For example, you might consider adding some specific examples of the types of leaves mentioned in the first line - "Golden maples" or "Crimson oaks

In [ ]:
external_termination.set()

# Wait for the team to finish.
await run


In [3]:
from autogen_agentchat.agents import AssistantAgent, UserProxyAgent
from autogen_agentchat.conditions import TextMentionTermination
from autogen_agentchat.teams import RoundRobinGroupChat
from autogen_agentchat.ui import Console
from autogen_ext.models.openai import OpenAIChatCompletionClient

 
assistant = AssistantAgent("assistant", model_client=model_client)
user_proxy = UserProxyAgent("user_proxy", input_func=input)  # Use input() to get user input from console.

# Create the termination condition which will end the conversation when the user says "APPROVE".
termination = TextMentionTermination("APPROVE", sources=["user_proxy"])

# Create the team.
team = RoundRobinGroupChat([assistant, user_proxy], termination_condition=termination)

# Run the conversation and stream to the console.
stream = team.run_stream(task="Write a 4-line poem about the ocean.")
# Use asyncio.run(...) when running in a script.
await Console(stream)

---------- user ----------
Write a 4-line poem about the ocean.
---------- assistant ----------
The vast blue waters stretch so far and wide
Calming sounds of waves, a soothing tide
Moonlight dances on the rippling sea
Peaceful serenity for you and me
---------- user_proxy ----------
Nah
---------- assistant ----------
Let's try again. 

Crashing waves upon the sandy shore
Ocean's roar, forever more
Deep blue mystery beneath the surface lies
Moments of solitude in the endless tides
---------- user_proxy ----------
Approve
---------- assistant ----------
I'm glad you liked it.

 TERMINATE
---------- user_proxy ----------
APPROVE


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a 4-line poem about the ocean.', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=49, completion_tokens=38), metadata={}, content='The vast blue waters stretch so far and wide\nCalming sounds of waves, a soothing tide\nMoonlight dances on the rippling sea\nPeaceful serenity for you and me', type='TextMessage'), UserInputRequestedEvent(source='user_proxy', models_usage=None, metadata={}, request_id='2b576553-4eba-4607-9dcf-5bc4ba76b1e3', content='', type='UserInputRequestedEvent'), TextMessage(source='user_proxy', models_usage=None, metadata={}, content='Nah', type='TextMessage'), TextMessage(source='assistant', models_usage=RequestUsage(prompt_tokens=98, completion_tokens=39), metadata={}, content="Let's try again. \n\nCrashing waves upon the sandy shore\nOcean's roar, forever more\nDeep blue mystery beneath the surface lies\nMoments of solitude in 

In [6]:

def approve():
    pass

# Create the primary agent.
primary_agent = AssistantAgent(
    "primary",
    model_client=model_client,
    system_message="You are a helpful AI assistant.",
)

# Create the critic agent with the approve function as a tool.
critic_agent = AssistantAgent(
    "critic",
    model_client=model_client,
    tools=[approve],  # Register the approve function as a tool.
    system_message="Provide constructive feedback. Use the approve tool to approve when all feedbacks are addressed.",
)


In [13]:
from autogen_agentchat.base import TerminationCondition, TerminatedException
from autogen_core import Component
from pydantic import BaseModel
from autogen_agentchat.messages import AgentEvent, ChatMessage, StopMessage, ToolCallExecutionEvent
from typing_extensions import Self
from typing import Sequence

class FunctionCallTerminationConfig(BaseModel):
    name:str
    

class FunctionCallTermination(TerminationCondition, Component[FunctionCallTerminationConfig]):
    
    component_config_schema = FunctionCallTerminationConfig
    
    def __init__(self, function_name: str) -> None:
        self._terminated = False
        self._function_name = function_name

    @property
    def terminated(self) -> bool:
        return self._terminated
    
    async def __call__(self, messages: Sequence[AgentEvent | ChatMessage]) -> StopMessage | None:
        if self._terminated:
            raise TerminatedException("Termination condition has already been reached")
        for message in messages:
            if isinstance(message, ToolCallExecutionEvent):
                for execution in message.content:
                    if execution.name == self._function_name:
                        self._terminated = True
                        return StopMessage(
                            content=f"Function '{self._function_name}' was executed.",
                            source="FunctionCallTermination",
                        )
        return None

    async def reset(self) -> None:
        self._terminated = False

    def _to_config(self) -> FunctionCallTerminationConfig:
        return FunctionCallTerminationConfig(
            function_name=self._function_name,
        )

    @classmethod
    def _from_config(cls, config: FunctionCallTerminationConfig) -> Self:
        return cls(
            function_name=config.function_name,
        )

In [14]:
functionCallTermination = FunctionCallTermination("approve")

team = RoundRobinGroupChat(
    [primary_agent, critic_agent],
    termination_condition=functionCallTermination,)

await Console(team.run_stream(task="Write a short poem about the fall season."))

---------- user ----------
Write a short poem about the fall season.
---------- primary ----------
Here's a short poem about fall:

Autumn leaves begin to play,
Dancing in breezes on their way.
Golden hues and crimson bright,
Filling hearts with warmth and delight.

Crisp air whispers through the trees,
As nature's final dance proceeds.
Fading summer's gentle sigh,
Echoes in the autumn sky.

Would you like me to revise anything or write a new one?
---------- critic ----------
[FunctionCall(id='call_std9rvod', arguments='{}', name='approve')]
---------- critic ----------
[FunctionExecutionResult(content='None', name='approve', call_id='call_std9rvod', is_error=False)]
---------- critic ----------
None


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a short poem about the fall season.', type='TextMessage'), TextMessage(source='primary', models_usage=RequestUsage(prompt_tokens=88, completion_tokens=82), metadata={}, content="Here's a short poem about fall:\n\nAutumn leaves begin to play,\nDancing in breezes on their way.\nGolden hues and crimson bright,\nFilling hearts with warmth and delight.\n\nCrisp air whispers through the trees,\nAs nature's final dance proceeds.\nFading summer's gentle sigh,\nEchoes in the autumn sky.\n\nWould you like me to revise anything or write a new one?", type='TextMessage'), ToolCallRequestEvent(source='critic', models_usage=RequestUsage(prompt_tokens=320, completion_tokens=12), metadata={}, content=[FunctionCall(id='call_std9rvod', arguments='{}', name='approve')], type='ToolCallRequestEvent'), ToolCallExecutionEvent(source='critic', models_usage=None, metadata={}, content=[FunctionExecutionResult(content='

In [29]:
from autogen_agentchat.conditions import MaxMessageTermination
assistant_agent = AssistantAgent(
    name="assistant_agent",
    system_message="You are a helpful assistant",
    model_client= model_client
)
agent_team = RoundRobinGroupChat([assistant_agent], termination_condition=MaxMessageTermination(max_messages=2))
await Console(agent_team.run_stream(task="Write a joke about CS go2 gamling?"))


---------- user ----------
Write a joke about CS go2 gamling?
---------- assistant_agent ----------
Why did the CS: Go player bring a ladder to the match?

Because he wanted to take his bets to the next level! (get it?)


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='Write a joke about CS go2 gamling?', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=30, completion_tokens=31), metadata={}, content='Why did the CS: Go player bring a ladder to the match?\n\nBecause he wanted to take his bets to the next level! (get it?)', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [30]:
team_state = await agent_team.save_state()

await agent_team.reset()

await Console(agent_team.run_stream(task="what was the last topic that we talk about"))

---------- user ----------
what was the last topic that we talk about
---------- assistant_agent ----------
This is the beginning of our conversation, so we haven't discussed anything yet. What would you like to talk about? I can summarize our conversation at the end if you'd like!


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='what was the last topic that we talk about', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=29, completion_tokens=38), metadata={}, content="This is the beginning of our conversation, so we haven't discussed anything yet. What would you like to talk about? I can summarize our conversation at the end if you'd like!", type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [31]:
print(team_state)

{'type': 'TeamState', 'version': '1.0.0', 'agent_states': {'assistant_agent': {'type': 'ChatAgentContainerState', 'version': '1.0.0', 'agent_state': {'type': 'AssistantAgentState', 'version': '1.0.0', 'llm_context': {'messages': [{'content': 'Write a joke about CS go2 gamling?', 'source': 'user', 'type': 'UserMessage'}, {'content': 'Why did the CS: Go player bring a ladder to the match?\n\nBecause he wanted to take his bets to the next level! (get it?)', 'thought': None, 'source': 'assistant_agent', 'type': 'AssistantMessage'}]}}, 'message_buffer': []}, 'RoundRobinGroupChatManager': {'type': 'RoundRobinManagerState', 'version': '1.0.0', 'message_thread': [{'source': 'user', 'models_usage': None, 'metadata': {}, 'content': 'Write a joke about CS go2 gamling?', 'type': 'TextMessage'}, {'source': 'assistant_agent', 'models_usage': {'prompt_tokens': 30, 'completion_tokens': 31}, 'metadata': {}, 'content': 'Why did the CS: Go player bring a ladder to the match?\n\nBecause he wanted to take 

In [32]:
await agent_team.load_state(team_state)

await Console(agent_team.run_stream(task="what was the last topic that we talk about"))

---------- user ----------
what was the last topic that we talk about


---------- assistant_agent ----------
Our last topic was talking about CS: GO and creating a joke about CS: GO 2 gaming.


TaskResult(messages=[TextMessage(source='user', models_usage=None, metadata={}, content='what was the last topic that we talk about', type='TextMessage'), TextMessage(source='assistant_agent', models_usage=RequestUsage(prompt_tokens=79, completion_tokens=22), metadata={}, content='Our last topic was talking about CS: GO and creating a joke about CS: GO 2 gaming.', type='TextMessage')], stop_reason='Maximum number of messages 2 reached, current message count: 2')

In [34]:
import json

with open("coding/team_state.json", "w") as f:
    json.dump(team_state, f)

In [ ]:
from autogen_ext.memory.chromadb